In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import itertools
%matplotlib inline

In [2]:
dados = pd.read_csv('mundo_transfermarkt_competicoes_brasileirao_serie_a.csv.gz',delimiter=',')

In [3]:
# Selecionando apenas dados de 2023
BR_2023 = dados[dados['ano_campeonato'] == 2023]
BR_2023.columns

Index(['ano_campeonato', 'data', 'rodada', 'estadio', 'arbitro', 'publico',
       'publico_max', 'time_mandante', 'time_visitante', 'tecnico_mandante',
       'tecnico_visitante', 'colocacao_mandante', 'colocacao_visitante',
       'valor_equipe_titular_mandante', 'valor_equipe_titular_visitante',
       'idade_media_titular_mandante', 'idade_media_titular_visitante',
       'gols_mandante', 'gols_visitante', 'gols_1_tempo_mandante',
       'gols_1_tempo_visitante', 'escanteios_mandante', 'escanteios_visitante',
       'faltas_mandante', 'faltas_visitante', 'chutes_bola_parada_mandante',
       'chutes_bola_parada_visitante', 'defesas_mandante', 'defesas_visitante',
       'impedimentos_mandante', 'impedimentos_visitante', 'chutes_mandante',
       'chutes_visitante', 'chutes_fora_mandante', 'chutes_fora_visitante'],
      dtype='object')

In [4]:
BR_2023 = BR_2023[['rodada','time_mandante', 'time_visitante', 'gols_mandante','gols_visitante', 'chutes_mandante',
                   'chutes_visitante']]

In [5]:
BR_2023.head()

,rodada,time_mandante,time_visitante,gols_mandante,gols_visitante,chutes_mandante,chutes_visitante
2660,2,Fluminense,Athletico-PR,2.0,0.0,14.0,10.0
2661,8,Botafogo,América-MG,2.0,0.0,18.0,4.0
2662,15,Internacional,Palmeiras,0.0,0.0,9.0,16.0
2663,15,Athletico-PR,EC Bahia,2.0,0.0,15.0,8.0
2664,27,Goiás,São Paulo,2.0,0.0,7.0,8.0


In [6]:
BR_2023 = BR_2023.sort_values(by="rodada")
BR_2023.reset_index(drop=True, inplace=True)

In [7]:
BR_2023[BR_2023.isna().any(axis = 1)]

,rodada,time_mandante,time_visitante,gols_mandante,gols_visitante,chutes_mandante,chutes_visitante
153,16,Flamengo,América-MG,1.0,1.0,NaN,NaN
159,16,Palmeiras,Fortaleza,3.0,1.0,NaN,NaN


In [8]:
valores = {159: {'chutes_mandante':14,      
'chutes_visitante':11},
153: {'chutes_mandante' :16,     
'chutes_visitante':16}
}


# Preencher os dados faltantes
for idx, valores in valores.items():
    BR_2023.loc[idx] = BR_2023.loc[idx].fillna(value=valores)

BR_2023.isna().sum()

rodada              0
time_mandante       0
time_visitante      0
gols_mandante       0
gols_visitante      0
chutes_mandante     0
chutes_visitante    0
dtype: int64

### Criando novas colunas

In [9]:
condicoes = [
    (BR_2023['gols_mandante'] - BR_2023['gols_visitante'] == 0),  # Se a diferença for 0
    (BR_2023['gols_mandante'] - BR_2023['gols_visitante'] > 0),   # Se a diferença for maior que 0
    (BR_2023['gols_mandante'] - BR_2023['gols_visitante'] < 0)    # Se a diferença for menor que 0
]

# Definindo os valores para cada condição
valores = [0, 1, 2]

# Criando a nova coluna com base nas condições
BR_2023['resultado'] = np.select(condicoes, valores)

In [10]:
# Função para calcular pontos
def calcular_pontos(row):
    if row['resultado'] == 1:  # Mandante venceu
        return 3, 0
    elif row['resultado'] == 2:  # Visitante venceu
        return 0, 3
    else:  # Empate
        return 1, 1

# Aplicando a função e criando novas colunas
BR_2023[['pontos_mandante', 'pontos_visitante']] = BR_2023.apply(calcular_pontos, axis=1, result_type='expand')

In [11]:
# Função para calcular pontos
def vitoria_empate_derrota(row):
    if row['resultado'] == 1:  # Mandante venceu
        return 1, 0, 0, 0, 0, 1
    elif row['resultado'] == 2:  # Visitante venceu
        return 0, 0, 1, 1, 0, 0
    else:  # Empate
        return 0, 1, 0, 0, 1, 0
    
BR_2023[['vitoria_mandante', 'empate_mandante','derrota_mandante','vitoria_visitante','empate_visitante','derrota_visitante']] = BR_2023.apply(vitoria_empate_derrota, axis=1, result_type='expand')

In [ ]:
times = ['América-MG', 'Athletico-PR', 'Atlético-MG',
 'Botafogo', 'Corinthians', 'Coritiba FC', 'Cruzeiro',
 'Cuiabá-MT', 'EC Bahia', 'Flamengo', 'Fluminense',
 'Fortaleza', 'Goiás', 'Grêmio', 'Internacional',
 'Palmeiras', 'RB Bragantino', 'Santos', 'São Paulo', 'Vasco da Gama']

In [13]:
# vitorias em casa e fora pelas equipes 
soma_vitorias_mandante = []
soma_vitorias_visitante = []
for time in lista:
    vitorias_mandante = int(BR_2023[BR_2023['time_mandante']== time]['vitoria_mandante'].sum())
    vitorias_visitante = int(BR_2023[BR_2023['time_visitante']== time]['vitoria_visitante'].sum())

    soma_vitorias_mandante.append(vitorias_mandante)
    soma_vitorias_visitante.append(vitorias_visitante)   

# total de vitorias de cada equipe
total_vitorias = np.array(soma_vitorias_mandante) + np.array(soma_vitorias_visitante)

In [14]:
# empates em casa e fora pelas equipes 
soma_empate_mandante = []
soma_empate_visitante = []
for time in lista:
    empate_mandante = int(BR_2023[BR_2023['time_mandante']== time]['empate_mandante'].sum())
    empate_visitante = int(BR_2023[BR_2023['time_visitante']== time]['empate_visitante'].sum())

    soma_empate_mandante.append(empate_mandante)
    soma_empate_visitante.append(empate_visitante)   

# total de empates de cada equipe
total_empate = np.array(soma_empate_mandante) + np.array(soma_empate_visitante)

In [15]:
# derrotas em casa e fora pelas equipes 
soma_derrota_mandante = []
soma_derrota_visitante = []
for time in lista:
    derrota_mandante = int(BR_2023[BR_2023['time_mandante']== time]['derrota_mandante'].sum())
    derrota_visitante = int(BR_2023[BR_2023['time_visitante']== time]['derrota_visitante'].sum())

    soma_derrota_mandante.append(derrota_mandante)
    soma_derrota_visitante.append(derrota_visitante)   

# total de derrotas de cada equipe
total_derrota = np.array(soma_derrota_mandante) + np.array(soma_derrota_visitante)

In [16]:
# Gols feitos em casa e fora pelas equipes 
soma_gols_mandante = []
soma_gols_visitante = []
for time in lista:
    gols_mandante = int(BR_2023[BR_2023['time_mandante']== time]['gols_mandante'].sum())
    gols_visitante = int(BR_2023[BR_2023['time_visitante']== time]['gols_visitante'].sum())

    soma_gols_mandante.append(gols_mandante)
    soma_gols_visitante.append(gols_visitante)   

# total de gols de cada equipe
total_gols = np.array(soma_gols_mandante) + np.array(soma_gols_visitante)

In [17]:
# Gols sofridos em casa e fora pelas equipes 
soma_sofridos_mandante = []
soma_sofridos_visitante = []
for time in lista:
    sofridos_mandante = int(BR_2023[BR_2023['time_mandante']== time]['gols_visitante'].sum())
    sofridos_visitante = int(BR_2023[BR_2023['time_visitante']== time]['gols_mandante'].sum())

    soma_sofridos_mandante.append(sofridos_mandante)
    soma_sofridos_visitante.append(sofridos_visitante)   


total_sofridos = np.array(soma_sofridos_mandante) + np.array(soma_sofridos_visitante)

In [18]:
# pontos feitos em casa e fora pelas equipes 
soma_pontos_mandante = []
soma_pontos_visitante = []
for time in lista:
    pontos_mandante = int(BR_2023[BR_2023['time_mandante']== time]['pontos_mandante'].sum())
    pontos_visitante = int(BR_2023[BR_2023['time_visitante']== time]['pontos_visitante'].sum())

    soma_pontos_mandante.append(pontos_mandante)
    soma_pontos_visitante.append(pontos_visitante)   


total_pontos = np.array(soma_pontos_mandante) + np.array(soma_pontos_visitante)

### Funções de cores

In [ ]:
cores = ['#006400','#228B22' ,'#006400','#0000FF' ,'#006400','#0000FF', '#FFFFFF', '#FF0000']
repeticoes = [4,2,1,3,1,3,2,4]
Lista_cores = list(itertools.chain.from_iterable(itertools.repeat(cor, n) for cor, n in zip(cores, repeticoes)))
# Função para adicionar as cores
def cores_classificacao(row):
    pos = row['Classificação']
    cor = Lista_cores[(pos-1)] 
    cor_texto = '#000000' if cor == '#FFFFFF' else '#FFFFFF'

    # Create an HTML circle with the classification number inside
    return f'<div style="border-radius: 50%; width: 20px; height: 20px; background-color: {cor}; display: flex; justify-content: center; align-items: center; color: {cor_texto};">{pos}</div>'


### Tabela Completa

In [ ]:
# Extraindo dados e criando um novo DataFrame
dict = {'Equipes': lista,
 'P': total_pontos,
 'V': total_vitorias,
 'E': total_empate,
 'D': total_derrota,
 'GF': total_gols,
 'GS': total_sofridos,
 'SG': np.array(total_gols) - np.array(total_sofridos)
}
Tabela_completa = pd.DataFrame(dict)

Tabela_completa.sort_values(by=['P','V','SG'], ascending=[False,False,False],inplace=True)

# Adicionar a coluna de classificação
Tabela_completa.insert(0, 'Classificação', range(1, len(Tabela_completa) + 1))

# Aplicando a função para colocar cores referentes as classificações de competições
Tabela_completa['Classificação'] = Tabela_completa.apply(cores_classificacao, axis=1)

# Mostrar a tabela
display(HTML(Tabela_completa.to_html(escape=False, index=False)))


Classificação,Equipes,P,V,E,D,GF,GS,SG
1,Palmeiras,70,20,10,8,64,33,31
2,Grêmio,68,21,5,12,63,56,7
3,Atlético-MG,66,19,9,10,52,32,20
4,Flamengo,66,19,9,10,56,42,14
5,Botafogo,64,18,10,10,58,37,21
6,RB Bragantino,62,17,11,10,49,35,14
7,Fluminense,56,16,8,14,51,47,4
8,Athletico-PR,56,14,14,10,51,43,8
9,Internacional,55,15,10,13,46,45,1
10,Fortaleza,54,15,9,14,45,44,1


### Tabela para mandantes

In [ ]:
dict = {'Equipes': times,
 'P': soma_pontos_mandante,
 'V': soma_vitorias_mandante,
 'E': soma_empate_mandante,
 'D': soma_derrota_mandante,
 'GF': soma_gols_mandante,
 'GS': soma_sofridos_mandante,
 'SG': np.array(soma_gols_mandante) - np.array(soma_sofridos_mandante)
}
Tabela_completa = pd.DataFrame(dict)

Tabela_completa.sort_values(by=['P','V','SG'], ascending=[False,False,False],inplace=True)

# Adicionar a coluna de classificação
Tabela_completa.insert(0, 'Classificação', range(1, len(Tabela_completa) + 1))

# Mostrar a classificação
display(HTML(Tabela_completa.to_html(index=False)))


### Tabela para visitantes

In [ ]:
dict = {'Equipes': times,
 'P': soma_pontos_visitante,
 'V': soma_vitorias_visitante,
 'E': soma_empate_visitante,
 'D': soma_derrota_visitante,
 'GF': soma_gols_visitante,
 'GS': soma_sofridos_visitante,
 'SG': np.array(soma_gols_visitante) - np.array(soma_sofridos_visitante)
}
Tabela_completa = pd.DataFrame(dict)

Tabela_completa.sort_values(by=['P','V','SG'], ascending=[False,False,False],inplace=True)

# Adicionar a coluna de classificação
Tabela_completa.insert(0, 'Classificação', range(1, len(Tabela_completa) + 1))

# Mostrar a classificação
display(HTML(Tabela_completa.to_html(index=False)))
